# Lab-1.8: ORPO 環境設置與數據準備

**實驗目標**: 準備 ORPO (Odds Ratio Preference Optimization) 訓練環境

**ORPO** 是比 DPO 更先進的對齊技術，主要創新點：
- **單階段訓練**: 無需預先 SFT，直接從偏好數據進行對齊
- **統一目標**: 同時優化 instruction following 和 preference alignment
- **效率提升**: 訓練時間和記憶體使用都大幅減少

## ORPO vs DPO

| 特性 | DPO | ORPO |
|------|-----|------|
| 訓練階段 | 2 (SFT + DPO) | 1 (ORPO) |
| 參考模型 | 需要 | 不需要 |
| 記憶體占用 | 2x | 1x |
| 訓練時間 | 100% | 50-60% |

---

## 步驟 1: 環境檢查與安裝

檢查 ORPO 所需的環境和套件。

In [ ]:
import torch
import sys
import os
import subprocess
import importlib

print(f'Python 版本: {sys.version}')
print(f'PyTorch 版本: {torch.__version__}')
print(f'CUDA 可用: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU 設備: {torch.cuda.get_device_name()}')
    print(f'GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
    print(f'當前記憶體使用: {torch.cuda.memory_allocated() / 1e9:.2f} GB')

# 設置隨機種子
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

print('\n🚀 ORPO 環境檢查完成')

In [ ]:
# 檢查並安裝必要套件
def install_and_import(package, import_name=None):
    if import_name is None:
        import_name = package
    
    try:
        importlib.import_module(import_name)
        print(f'✅ {package} 已安裝')
    except ImportError:
        print(f'⚠️  安裝 {package}...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
        print(f'✅ {package} 安裝完成')

# 檢查必要套件
required_packages = [
    'transformers',
    'peft', 
    'trl',
    'datasets',
    'bitsandbytes',
    'accelerate',
    'matplotlib',
    'seaborn',
    'scikit-learn'
]

for package in required_packages:
    install_and_import(package)

In [ ]:
# 導入所需庫並檢查版本
import transformers
import peft
import trl
import datasets
import bitsandbytes
import accelerate
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

print('=== 套件版本資訊 ===')
print(f'🤗 Transformers: {transformers.__version__}')
print(f'🎯 PEFT: {peft.__version__}')
print(f'🚀 TRL: {trl.__version__}')
print(f'📊 Datasets: {datasets.__version__}')
print(f'⚡ Accelerate: {accelerate.__version__}')

# 檢查 TRL 是否支援 ORPO
try:
    from trl import ORPOTrainer, ORPOConfig
    print('✅ TRL 支援 ORPO')
except ImportError:
    print('⚠️  TRL 版本可能不支援 ORPO，將使用自定義實現')

print('\n✅ ORPO 環境準備完成！')

## 步驟 2: 偏好數據集準備

ORPO 直接使用偏好數據進行訓練，無需額外的 instruction dataset。

In [ ]:
from datasets import load_dataset, Dataset

# 載入偏好數據集
print('📥 載入偏好數據集...')

try:
    # 載入 Anthropic HH-RLHF 數據集
    dataset = load_dataset('Anthropic/hh-rlhf', split='train[:500]')  # 載入較大樣本
    print(f'✅ 成功載入 HH-RLHF 數據集，樣本數: {len(dataset)}')
    
except Exception as e:
    print(f'❌ HH-RLHF 載入失敗: {e}')
    print('使用增強版模擬數據...')
    
    # 創建更豐富的模擬偏好數據
    mock_data = {
        'chosen': [
            'Human: 請解釋什麼是機器學習?\n\nAssistant: 機器學習是人工智能的一個重要分支，它讓計算機能夠從數據中自動學習並改進性能，而無需明確編程。核心概念包括：\n\n1. **學習過程**: 通過分析大量數據來識別模式和規律\n2. **算法類型**: 包括監督學習、無監督學習和強化學習\n3. **應用領域**: 廣泛應用於圖像識別、自然語言處理、推薦系統等\n\n機器學習的價值在於能夠處理複雜的數據模式，為決策提供智能支持。',
            
            'Human: 如何開始學習程式設計?\n\nAssistant: 學習程式設計是一個循序漸進的過程，建議按以下步驟進行：\n\n**基礎階段**：\n1. 選擇適合的入門語言（如 Python）\n2. 學習基本語法和概念\n3. 練習簡單的邏輯問題\n\n**進階階段**：\n1. 學習數據結構和算法\n2. 做實際項目練習\n3. 閱讀他人代碼\n\n**持續成長**：\n1. 參與開源專案\n2. 學習軟體工程實踐\n3. 保持學習新技術的習慣\n\n記住，程式設計重在實作，多寫多練是關鍵。',
            
            'Human: 健康飲食的基本原則是什麼?\n\nAssistant: 健康飲食應遵循以下科學原則：\n\n**營養均衡**：\n- 碳水化合物：提供能量，選擇全穀類\n- 蛋白質：維持肌肉，選擇優質來源\n- 脂肪：選擇不飽和脂肪，避免反式脂肪\n- 維生素礦物質：多吃蔬果補充\n\n**飲食習慣**：\n- 規律用餐，避免暴飲暴食\n- 控制份量，適量攝取\n- 多喝水，少喝含糖飲料\n- 減少加工食品\n\n**個人化調整**：\n根據年齡、性別、活動量調整飲食計劃。建議諮詢營養師制定個人化方案。',
            
            'Human: 如何提高工作效率?\n\nAssistant: 提高工作效率需要系統性方法：\n\n**時間管理**：\n1. 使用番茄工作法：25分鐘專注 + 5分鐘休息\n2. 優先級排序：先做重要且緊急的任務\n3. 時間塊規劃：為不同任務分配固定時間\n\n**環境優化**：\n1. 整理工作空間，減少干擾\n2. 使用高效工具和軟體\n3. 建立標準化流程\n\n**個人習慣**：\n1. 保持健康作息\n2. 定期休息和運動\n3. 持續學習新技能\n\n**團隊協作**：\n1. 清晰溝通，減少誤解\n2. 有效授權和分工\n3. 定期回顧和改進\n\n關鍵是找到適合自己的方法並持續執行。',
            
            'Human: 分析人工智能對未來就業的影響\n\nAssistant: 人工智能對就業市場的影響是複雜且多面向的：\n\n**潛在挑戰**：\n1. **自動化替代**：重複性工作面臨被取代風險\n2. **技能鴻溝**：需要新技能以適應變化\n3. **過渡期痛苦**：短期內可能造成失業\n\n**新機遇**：\n1. **新職業創造**：AI工程師、數據科學家等\n2. **工作增強**：AI作為工具提升人類能力\n3. **創意領域擴展**：更多時間專注創新工作\n\n**應對策略**：\n1. **終身學習**：持續更新技能\n2. **跨領域能力**：培養AI無法替代的軟技能\n3. **政策支持**：政府需制定轉型政策\n\n總體而言，AI更可能是工作轉型而非完全替代，關鍵在於主動適應。'
        ],
        'rejected': [
            'Human: 請解釋什麼是機器學習?\n\nAssistant: 機器學習就是讓機器變聰明，能夠自己學東西。',
            
            'Human: 如何開始學習程式設計?\n\nAssistant: 學程式就是寫code，買本書看看就會了。',
            
            'Human: 健康飲食的基本原則是什麼?\n\nAssistant: 健康飲食就是少吃多動，不要吃垃圾食品。',
            
            'Human: 如何提高工作效率?\n\nAssistant: 工作效率就是要快一點，多做一些事情。',
            
            'Human: 分析人工智能對未來就業的影響\n\nAssistant: AI會搶走很多工作，大家要小心。'
        ]
    }
    
    dataset = Dataset.from_dict(mock_data)
    print(f'✅ 創建模擬數據集，樣本數: {len(dataset)}')

print(f'\n數據集欄位: {list(dataset.features.keys())}')
print(f'數據集大小: {len(dataset)}')

In [ ]:
# 探索數據集內容
print('=== 偏好數據樣本探索 ===')

for i in range(min(2, len(dataset))):
    sample = dataset[i]
    print(f'\n📝 樣本 {i+1}:')
    
    if 'chosen' in sample:
        print('✅ 偏好回應 (chosen):')
        chosen_text = sample['chosen']
        print(chosen_text[:300] + '...' if len(chosen_text) > 300 else chosen_text)
        print()
        
    if 'rejected' in sample:
        print('❌ 非偏好回應 (rejected):')
        rejected_text = sample['rejected']
        print(rejected_text[:300] + '...' if len(rejected_text) > 300 else rejected_text)
        print()
    
    print('-' * 80)

## 步驟 3: 數據預處理與格式化

將數據轉換為 ORPO 訓練所需的格式。

In [ ]:
def extract_prompt_and_response(text):
    """從對話文本中提取提示和回應"""
    if 'Human:' in text and 'Assistant:' in text:
        parts = text.split('Assistant:')
        if len(parts) >= 2:
            prompt = parts[0].replace('Human:', '').strip()
            response = parts[1].strip()
            return prompt, response
    return None, None


def preprocess_orpo_dataset(dataset):
    """預處理數據集為 ORPO 格式"""
    processed_data = []
    
    for sample in dataset:
        # 提取 chosen 和 rejected 的 prompt 和 response
        chosen_prompt, chosen_response = extract_prompt_and_response(sample['chosen'])
        rejected_prompt, rejected_response = extract_prompt_and_response(sample['rejected'])
        
        # 確保兩個回應使用相同的 prompt
        if chosen_prompt and chosen_response and rejected_response:
            # ORPO 格式: prompt, chosen, rejected
            processed_data.append({
                'prompt': chosen_prompt,
                'chosen': chosen_response,
                'rejected': rejected_response
            })
    
    return processed_data


# 預處理數據
print('🔄 預處理 ORPO 數據...')
processed_data = preprocess_orpo_dataset(dataset)

print(f'✅ 處理完成，有效樣本數: {len(processed_data)}')

# 顯示處理後的樣本
if processed_data:
    print('\n=== 處理後樣本示例 ===')
    sample = processed_data[0]
    print(f'📝 Prompt: {sample["prompt"]}\n')
    print(f'✅ Chosen: {sample["chosen"][:200]}...')
    print(f'❌ Rejected: {sample["rejected"][:200]}...')

## 步驟 4: ORPO 損失函數預覽

實現並測試 ORPO 核心損失函數。

In [ ]:
import torch.nn.functional as F

def compute_log_probs(model, input_ids, attention_mask, labels, tokenizer):
    """計算序列的對數概率"""
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    
    # 計算每個 token 的對數概率
    log_probs = F.log_softmax(logits, dim=-1)
    
    # 收集目標 token 的對數概率
    target_log_probs = torch.gather(log_probs[:, :-1], dim=-1, 
                                   index=labels[:, 1:].unsqueeze(-1)).squeeze(-1)
    
    # 僅計算非 padding token 的平均對數概率
    mask = (labels[:, 1:] != tokenizer.pad_token_id).float()
    sequence_log_prob = (target_log_probs * mask).sum(dim=1) / mask.sum(dim=1)
    
    return sequence_log_prob


def compute_odds_ratio_loss(chosen_log_probs, rejected_log_probs):
    """計算 ORPO 的 Odds Ratio 損失"""
    # 計算 log odds
    chosen_log_odds = chosen_log_probs - torch.log1p(-torch.exp(chosen_log_probs.clamp(max=0)))
    rejected_log_odds = rejected_log_probs - torch.log1p(-torch.exp(rejected_log_probs.clamp(max=0)))
    
    # Odds Ratio 損失
    log_odds_ratio = chosen_log_odds - rejected_log_odds
    loss = -F.logsigmoid(log_odds_ratio).mean()
    
    return loss, log_odds_ratio.mean()


def orpo_loss(model, batch, tokenizer, lambda_or=0.5):
    """
    ORPO 損失函數
    
    Args:
        model: 訓練中的模型
        batch: 包含 prompt, chosen, rejected 的批次
        tokenizer: tokenizer
        lambda_or: Odds Ratio 損失權重
    
    Returns:
        total_loss, metrics
    """
    # 1. SFT 損失 (標準語言模型損失，僅在 chosen 上)
    chosen_input_ids = batch['chosen_input_ids']
    chosen_attention_mask = batch['chosen_attention_mask']
    
    outputs = model(input_ids=chosen_input_ids, attention_mask=chosen_attention_mask, 
                   labels=chosen_input_ids)
    sft_loss = outputs.loss
    
    # 2. Odds Ratio 損失
    chosen_log_probs = compute_log_probs(model, batch['chosen_input_ids'], 
                                        batch['chosen_attention_mask'], 
                                        batch['chosen_input_ids'], tokenizer)
    
    rejected_log_probs = compute_log_probs(model, batch['rejected_input_ids'],
                                          batch['rejected_attention_mask'],
                                          batch['rejected_input_ids'], tokenizer)
    
    or_loss, log_odds_ratio = compute_odds_ratio_loss(chosen_log_probs, rejected_log_probs)
    
    # 3. 總損失
    total_loss = sft_loss + lambda_or * or_loss
    
    metrics = {
        'total_loss': total_loss.item(),
        'sft_loss': sft_loss.item(),
        'or_loss': or_loss.item(),
        'log_odds_ratio': log_odds_ratio.item(),
        'lambda_or': lambda_or
    }
    
    return total_loss, metrics


print('✅ ORPO 損失函數實現完成')
print('\n🔬 ORPO 核心概念:')
print('• SFT Loss: 確保模型基本的指令遵循能力')
print('• Odds Ratio Loss: 優化偏好對齊，提升 chosen 相對 rejected 的機率')
print('• Lambda 平衡: 控制兩個損失的相對重要性')
print('• 單階段訓練: 同時優化兩個目標，無需分階段')

## 步驟 5: 實驗設置總結

總結 ORPO 環境設置完成情況。

In [ ]:
# 實驗設置總結
print('=== ORPO 環境設置總結 ===')
print(f'✅ PyTorch 版本: {torch.__version__}')
print(f'✅ GPU 可用: {torch.cuda.is_available()}')
print(f'✅ 數據樣本: {len(processed_data) if processed_data else 0}')
print(f'✅ ORPO 損失函數: 已實現')

if processed_data:
    # 保存數據集
    output_dir = './orpo_data'
    os.makedirs(output_dir, exist_ok=True)
    
    # 轉換為 Dataset 並保存
    orpo_dataset = Dataset.from_list(processed_data)
    orpo_dataset.save_to_disk(output_dir)
    
    print(f'💾 數據集已保存至: {output_dir}')

print('\n🚀 ORPO 環境設置完成！')
print('下一步: 執行 02-ORPO_Training.ipynb 開始單階段對齊訓練')

print('\n🔬 ORPO 技術優勢:')
print('• 單階段訓練：無需 SFT + DPO 兩階段')
print('• 記憶體高效：無需參考模型，記憶體使用減半')
print('• 訓練簡化：統一損失函數，超參數調優更容易')
print('• 效果更佳：理論上比 DPO 有更好的對齊效果')

# 清理 GPU 記憶體
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f'\n💾 GPU 記憶體已清理，當前使用: {torch.cuda.memory_allocated() / 1e9:.2f} GB')